In [ ]:
# !pip install keras_cv_attention_models

In [1]:
import os
from tensorflow import keras
import losses, train, GhostFaceNets
import tensorflow as tf
import keras_cv_attention_models

2024-04-21 22:00:36.463174: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-21 22:00:36.463194: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-04-21 22:00:38.267089: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-21 22:00:38.267366: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-21 22:00:38.267440: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.s

In [2]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
print(tf.config.list_physical_devices('GPU'))

[]


In [3]:
# Remove the below for better accuracies and keep it for faster training
keras.mixed_precision.set_global_policy("mixed_float16")

The dtype policy mixed_float16 may run slowly because this machine does not have a GPU. Only Nvidia GPUs with compute capability of at least 7.0 run quickly with mixed_float16.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


GhostFaceNetV1

In [4]:
# ms1m-retinaface-t1 (MS1MV3) dataset
data_basic_path = 'datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/faces_emore_112x112_folders'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

# (VN-Celeb) dataset
data_path = 'datasets/vn_celeb_112x112_folders'
eval_paths = ['datasets/vn_celeb/lfw.bin', 'datasets/vn_celeb/cfp_fp.bin', 'datasets/vn_celeb/agedb_30.bin']

In [5]:
#GhostFaceNetV1
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

2024-04-21 23:02:38.512892: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


>>>> Change BatchNormalization momentum and epsilon default value.
>>>> Convert ReLU: activation --> activation
>>>> Convert ReLU: activation_1 --> activation_1
>>>> Convert ReLU: activation_2 --> activation_2
>>>> Convert ReLU: activation_3 --> activation_3
>>>> Convert ReLU: activation_4 --> activation_4
>>>> Convert ReLU: activation_5 --> activation_5
>>>> Convert ReLU: activation_6 --> activation_6
>>>> Convert ReLU: activation_7 --> activation_7
>>>> Convert ReLU: activation_8 --> activation_8
>>>> Convert ReLU: activation_9 --> activation_9
>>>> Convert ReLU: activation_11 --> activation_11
>>>> Convert ReLU: activation_12 --> activation_12
>>>> Convert ReLU: activation_13 --> activation_13
>>>> Convert ReLU: activation_15 --> activation_15
>>>> Convert ReLU: activation_16 --> activation_16
>>>> Convert ReLU: activation_17 --> activation_17
>>>> Convert ReLU: activation_18 --> activation_18
>>>> Convert ReLU: activation_19 --> activation_19
>>>> Convert ReLU: activation_20 --> ac

In [6]:
# Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

# Strides of 1
# tt = train.Train(data_path, eval_paths=eval_paths,
#     save_path='ghostnetv1_w1.3_s1.h5',
#     basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
#     batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

>>>> L2 regularizer value from basic_model: 0.00025


In [11]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 2, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 2},
]
tt.train(sch, 0)

>>>> Init type by loss function name...
>>>> Train arcface...
>>>> Init softmax dataset...
>>>> Image length: 23105, Image class length: 23105, classes: 1020
>>>> Use specified optimizer: <keras.optimizer_v2.gradient_descent.SGD object at 0x7c90d47f2130>
>>>> Add L2 regularizer to model output layer, output_weight_decay = 0.000500
>>>> Add arcface layer, arc_kwargs={'loss_top_k': 1, 'append_norm': False, 'partial_fc_split': 0, 'name': 'arcface'}, vpl_kwargs={'vpl_lambda': 0.15, 'start_iters': -180, 'allowed_delta': 200}...
>>>> loss_weights: {'arcface': 1}
Epoch 1/2

Learning rate for iter 1 is 0.10000000149011612, global_iterNum is 0


2024-04-21 23:04:25.122410: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-04-21 23:04:25.221660: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-04-21 23:04:25.263161: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-04-21 23:04:25.263270: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.
2024-04-21 23:04:25.288371: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 51380224 exceeds 10% of free system memory.


  3/180 [..............................] - ETA: 1:27:33 - loss: 32.4180 - accuracy: 0.0026

KeyboardInterrupt: 

GhostFaceNetV2

In [8]:
# ms1m-retinaface-t1 (MS1MV3) dataset
data_basic_path = 'datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/faces_emore_112x112_folders'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [9]:
#GhostFaceNetV2
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

>>>> Change BatchNormalization momentum and epsilon default value.
>>>> Convert ReLU: stack4_se_relu --> stack4_se_prelu
>>>> Convert ReLU: stack5_se_relu --> stack5_se_prelu
>>>> Convert ReLU: stack10_se_relu --> stack10_se_prelu
>>>> Convert ReLU: stack11_se_relu --> stack11_se_prelu
>>>> Convert ReLU: stack12_se_relu --> stack12_se_prelu
>>>> Convert ReLU: stack14_se_relu --> stack14_se_prelu
>>>> Convert ReLU: stack16_se_relu --> stack16_se_prelu


TypeError: backbones.ghostv2.GhostNetV2() got multiple values for keyword argument 'stem_strides'

In [ ]:
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

CosFaceLoss

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.CosFaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.CosFaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

Subcenter ArcFace Loss

In [ ]:
# ms1m-retinaface-t1 (MS1MV3) dataset
data_basic_path = 'datasets/ms1m-retinaface-t1'
data_path = data_basic_path + '_112x112_folders'
eval_paths = [os.path.join(data_basic_path, ii) for ii in ['lfw.bin', 'cfp_fp.bin', 'agedb_30.bin']]

# (MS1MV2) dataset
data_path = 'datasets/faces_emore_112x112_folders'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [ ]:
# GhostFaceNetV1
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

In [ ]:
# GhostFaceNetV2
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

In [ ]:
#GhostFaceNetV1
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s1_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
#GhostFaceNetV2
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s2_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s1_topk.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
""" First, Train with `lossTopK = 3` """
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer, "lossTopK": 3},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50, "lossTopK": 3},
]
tt.train(sch, 0)

In [ ]:
""" Then drop non-dominant subcenters and high-confident noisy data, which is `>75 degrees` """
import data_drop_top_k
# data_drop_top_k.data_drop_top_k('./checkpoints/TT_mobilenet_topk_bs256.h5', '/datasets/faces_casia_112x112_folders/', limit=20)
new_data_path = data_drop_top_k.data_drop_top_k(tt.model, tt.data_path)

In [ ]:
""" Train with the new dataset again, this time `lossTopK = 1` """
tt.reset_dataset(new_data_path)

At this point you can continue training the model or start training again and considering the generated dataset as a new dataset

In [ ]:
# Continue training
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)

In [ ]:
# Train the models from scatch using the new generated dataset

# New Cleaned Dataset
data_path = 'Path_To_New_Generated_Dataset'
eval_paths = ['datasets/faces_emore/lfw.bin', 'datasets/faces_emore/cfp_fp.bin', 'datasets/faces_emore/agedb_30.bin']

In [ ]:
# GhostFaceNetV1
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv1", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, scale=True, use_bias=True, strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

In [ ]:
# GhostFaceNetV2
# Strides of 2
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

# Strides of 1
basic_model = GhostFaceNets.buildin_models("ghostnetv2", dropout=0, emb_shape=512, output_layer='GDC', bn_momentum=0.9, bn_epsilon=1e-5, stem_strides=1)
basic_model = GhostFaceNets.add_l2_regularizer_2_model(basic_model, weight_decay=5e-4, apply_to_batch_normal=False)
basic_model = GhostFaceNets.replace_ReLU_with_PReLU(basic_model)

In [ ]:
# GhostFaceNetV1
# Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

# Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv1_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
# GhostFaceNetV2
#Strides of 2
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s2.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

#Strides of 1
tt = train.Train(data_path, eval_paths=eval_paths,
    save_path='ghostnetv2_w1.3_s1.h5',
    basic_model=basic_model, model=None, lr_base=0.1, lr_decay=0.5, lr_decay_steps=45, lr_min=1e-5,
    batch_size=128, random_status=0, eval_freq=1, output_weight_decay=1)

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.1, momentum=0.9)
sch = [
    {"loss": losses.ArcfaceLoss(scale=32), "epoch": 1, "optimizer": optimizer},
    {"loss": losses.ArcfaceLoss(scale=64), "epoch": 50},
]
tt.train(sch, 0)